<h1>
<center>
Midterm 2
</center>
</h1>
<div class=h1_cell>
<p>
There are 2 parts on the midterm, 50 points each. Each part has 3 questions. There is also an extra credit problem at the end.
<p>
We will use the Titanic table for this midterm. I have a wrangled version of it on google sheets that I set up for you.
<p>
</div>

In [0]:
import pandas as pd


titanic_table = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR2XbLnUuYlxaFMdS8_bBX3iKUIDEii6Lg5Rxesf-Oh8a6z8-vAN6UDGejaOrBg5130h4O_dLkecKWQ/pub?output=csv')



In [0]:
pd.set_option('display.max_columns', None)

In [3]:
titanic_table.head(1)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,no_age,filled_age,emb_C,emb_Q,emb_S,emb_nan,age_bin,age_Child,age_Adult,age_Senior,sex_female,sex_male,ok_child,pclass_1,pclass_2,pclass_3,pclass_nan
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,0,22.0,0,0,1,0,Child,1,0,0,0,1,0,0,0,1,0


In [0]:
!rm library_w19_week7.py

In [5]:
from google.colab import files
files.upload()

Saving library_w19_week7.py to library_w19_week7.py


{'library_w19_week7.py': b'import random\nimport numpy as np\nfrom functools import reduce\nfrom types import SimpleNamespace\nimport pandas as pd\n\ndef predictor_case(row, pred, target):\n  case_dict = {(0,0): \'true_negative\', (1,1): \'true_positive\', (0,1): \'false_negative\', (1,0): \'false_positive\'}\n  actual = row[target]\n  prediction = row[pred]\n  case = case_dict[(prediction, actual)]\n  return case\n\ndef accuracy(cases):\n  if \'true_positive\' in cases:\n    tp = cases[\'true_positive\']\n  else: tp = 0\n  if \'true_negative\' in cases: \n    tn = cases[\'true_negative\']\n  else: tn = 0\n  if \'false_positive\' in cases: \n    fp = cases[\'false_positive\']\n  else: fp = 0\n  if \'false_negative\' in cases:\n    fn = cases[\'false_negative\']\n  else: fn = 0\n  total = tp + tn + fp + fn\n  if (total > 0):\n    return (tp + tn)/(tp+tn+fp+fn)\n  else: return 0\n\ndef f1(cases):\n\n   #the heart of the matrix\n  if \'true_positive\' in cases:\n   tp = cases[\'true_posit

In [6]:
from library_w19_week7 import *

%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 compute_training	 euclidean_distance	 f1	 find_best_splitter	 forest_builder	 
forest_scores	 generate_table	 gig	 gini	 informedness	 k_fold	 k_fold_random	 knn	 knn_tester	 
oob_forest_scores	 oob_vote_taker	 path_id	 predictor_case	 probabilities	 produce_scores	 reorder_paths	 tree_predictor	 verify_unique	 
vote_taker	 


<h2>Columns used in all questions</h2>

We will use 14 columns from the table to make predictions on `Survived`.

In [0]:
usable_columns = [
 'emb_C',
 'emb_Q',
 'emb_S',
 'emb_nan',
 'age_Child',
 'age_Adult',
 'age_Senior',
 'no_age',
 'ok_child',
 'sex_female',
 'pclass_1',
 'pclass_2',
 'pclass_3',
 'pclass_nan'
]

target = 'Survived'

<h1>
Part 1. Explore random forest behavior
</h1>
<div class=h1_cell>
<p>
For prediction purposes, we have been treating a forest as a single predictor. It does call on its trees to get a prediction, but we don't see that. All we see is the final prediction. I'd like to dig deeper into individual tree behavior. I'll ask you to complete a set of programming problems to do this.
</div>

<h2>
Problem 1. Build a matrix of tree predictions (10 points)
</h2>
<div class=h1_cell>
<p>
The matrix will be a list of lists. Let's say the forest we are working with has 11 trees. Then the first list will be the predictions of the 11 trees for row 0 of the Titanic table, i.e., it will be a list of 11 binary values. The next list will be the 11 predictions for row 1 of the Titanic table. Given that we have 891 rows in the Titanic table, we will have 891 lists in our outer list.
<p>
Let's say I store my matrix in `all_trees`. Then `all_trees[i][j]` will represent the vote of the jth tree for row i.
  <p>
    Here is what you should see for the first 10 rows.
<p>
<pre>
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]]
 </pre>

</div>

<h2>
Here is forest to test on
</h2>
<div class=h1_cell>
<p>
  But first set random seeds so you get same results as mine.
</div>

In [0]:
import random

rng = np.random.RandomState(24)  #Will pass as arg to pandas sample method
random.seed(2000)

In [10]:
forest11 = forest_builder(titanic_table, usable_columns, target, hypers={'total-trees':11, 'random-state':rng})
len(forest11)

11

Now build the matrix and check your results against mine above.

In [11]:
all_trees = []
for i in range (len(titanic_table)):
  rowpred = [];
  for n,m in enumerate(forest11):
    rowpred.append(tree_predictor(titanic_table.loc[i], m))
  all_trees.append(rowpred)
  
all_trees[:10]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]]

<h1>
Problem 2: The most correct tree (10 points)
</h1>
<div class=h1_cell>
<p>
Produce an ordered list, from most correct to least correct, of the 11 trees taken over the 891 rows. A tree is correct if its prediction matches the `Survived` column in the target row. Here are my results.
  <pre>
[(2, 707),
 (1, 701),
 (10, 701),
 (4, 674),
 (6, 643),
 (7, 637),
 (3, 613),
 (0, 595),
 (5, 581),
 (8, 577),
 (9, 574)]
 </pre>
  The winner is tree2. It was correct 707 out of 891 times.
</div>

In [20]:
correctness = {}
# initialize the dict to 0 for each tree's correctness count
for i in range(len(forest11)):
  correctness[i] = 0
for i in range(len(titanic_table)):
  row_preds = all_trees[i]
  row_in_table = titanic_table.loc[i]
  target_value = row_in_table[target]
  for j in range(len(row_preds)):
    if (row_preds[j] == target_value):
      correctness[j]+=1

sorted(correctness.items(), key=lambda pair: pair[1], reverse=True)

[(2, 707),
 (1, 701),
 (10, 701),
 (4, 674),
 (6, 643),
 (7, 637),
 (3, 613),
 (0, 595),
 (5, 581),
 (8, 577),
 (9, 574)]

<h1>
Problem 3: voting blocks (30 points)
</h1>
<div class=h1_cell>
<p>
Taking trees pairwise, produce an ordered list of which trees vote the same (right or wrong) most often.
  <p>
    For each specific row:
    <pre>
 1. if (tree1,tree2) vote the same then I record that.
 2. if (tree1,tree3) vote the same then I record that.
 etc
</pre>
I do that for the entire table. Here is first part of sorted list I got for forest11.
<pre>
[[((5, 8), 887),
 ((2, 10), 863),
 ((7, 10), 815),
 ((1, 4), 810),
 ((3, 10), 797),
 ((2, 7), 787),
 ((1, 6), 785),
 ((4, 6), 780),
 ((2, 3), 769),
 ((7, 9), 754)]
</pre>
  So tree5 and tree8 vote the same (right or wrong) 887 out of 891 times.
</div>

In [41]:
from itertools import combinations_with_replacement
#create combinations of the trees
combos = list(combinations_with_replacement((0,1,2,3,4,5,6,7,8,9,10),2))
voting_blocks = {}
#add them to a dict with the initialized value as 0 
for i in range(len(combos)):
  voting_blocks[combos[i]] = 0
#delete the values where its the same row, i.e. {((0,0):0)}
for i in range(len(forest11)):
  del voting_blocks[(i,i)]
# build dict
for i in range(len(titanic_table)):
  row = all_trees[i]
  for com in voting_blocks:
    if(row[com[0]] == row[com[1]]):
      voting_blocks[com]+=1

sorted(voting_blocks.items(), key=lambda pair: pair[1], reverse=True)

[((5, 8), 887),
 ((2, 10), 863),
 ((7, 10), 815),
 ((1, 4), 810),
 ((3, 10), 797),
 ((2, 7), 787),
 ((1, 6), 785),
 ((4, 6), 780),
 ((2, 3), 769),
 ((7, 9), 754),
 ((1, 10), 747),
 ((3, 7), 721),
 ((1, 2), 719),
 ((6, 10), 711),
 ((2, 6), 695),
 ((3, 6), 685),
 ((4, 10), 680),
 ((9, 10), 680),
 ((3, 9), 678),
 ((5, 7), 675),
 ((1, 7), 671),
 ((7, 8), 671),
 ((2, 9), 668),
 ((0, 10), 661),
 ((2, 4), 656),
 ((0, 2), 655),
 ((1, 3), 653),
 ((2, 8), 651),
 ((2, 5), 647),
 ((6, 7), 643),
 ((3, 5), 633),
 ((3, 8), 629),
 ((5, 10), 627),
 ((8, 10), 623),
 ((0, 9), 610),
 ((4, 5), 610),
 ((4, 8), 606),
 ((4, 7), 604),
 ((6, 9), 598),
 ((3, 4), 596),
 ((0, 7), 585),
 ((8, 9), 568),
 ((0, 3), 567),
 ((6, 8), 565),
 ((5, 9), 564),
 ((5, 6), 561),
 ((1, 5), 553),
 ((1, 8), 549),
 ((1, 9), 546),
 ((0, 4), 538),
 ((0, 5), 521),
 ((0, 1), 517),
 ((0, 8), 517),
 ((0, 6), 505),
 ((4, 9), 493)]

It looks like for 891 rows, trees 5 and 8 vote the same in 887 of them. So they vote the same except for 4 rows.

<h1>
Part 2. Explore knn behavior
</h1>
<div class=h1_cell>
<p>
I'd like more information on how the rows are selected and how they vote.
  <p>
    Note for each of these problems I do not care what the actual prediction is. I am not computing accuracy but instead the internal working of the rows in voting.
    <p>
      <p>
Given the constrained time of the midterm, I am going to slice off 100 rows and just work with that.
</div>

In [0]:
small_titanic = titanic_table[:100]  #use this slice in your code to cut down on computing time

<h1>
Problem 4: A row's top-k counts (20 points)
</h1>
<div class=h1_cell>
<p>
For each row, count how many times it made the top-k. Store your answer in top_list in sorted fashion. Here are the first part of my results
  <pre>
[(0, 30),
 (12, 28),
 (4, 27),
 (13, 27),
 (37, 27),
 (8, 19),
 (29, 19),
 (45, 19),
 (51, 19),
 (59, 19)]
 </pre>
  So row 0 was among the top-k chosen 30 times out of 100 rows.
</div>

<h3>Hint</h3>

For this problem, and really most of the problems dealing with KNN, I started with the knn function and then modified it to do what is called for. For this problem, what is called for is a modified version of knn that does not return a prediction but instead the k rows selected to vote, i.e. the top k.
<p>
  Once I had the modified function, I called it in a loop that iterated over all the row indices of the Titanic table. The loop body updated a global data structure. At the end I got it in sorted list form and was finished.

In [0]:
def knn_topk(row_index, table, columns,k, target):
  length = len(table) # get the length of the table
  distance = {} # an empty dictionary to hold the distances between the row index and all the other rows in table
  t= table[columns]
  distance = euclidean_distance(t.iloc[[row_index]], t)
  # sort the distances   
  sorted_dist = sorted(distance.items(), key=operator.itemgetter(1)) # returns a sorted list of tuples 
  top_k = []
  #find the top k and increment their counts
  for key,item in enumerate(sorted_dist):
    if (key > k):
      break
    top_k.append(item[0])
  return top_k 

In [76]:
topk = {}
for i in range(len(small_titanic)):
  topk[i] = 0
for i in range(len(small_titanic)):
  top_k = knn_topk(i, small_titanic, usable_columns, 10, target)
  # top_k is a list of the indices of the top k rows
  for ind in top_k:
    topk[ind]+=1
sorted(topk.items(), key=lambda pair: pair[1], reverse=True)

[(0, 32),
 (12, 28),
 (4, 27),
 (13, 27),
 (37, 27),
 (51, 27),
 (2, 21),
 (23, 20),
 (8, 19),
 (29, 19),
 (45, 19),
 (59, 19),
 (67, 19),
 (69, 18),
 (20, 17),
 (3, 16),
 (5, 16),
 (10, 16),
 (21, 16),
 (74, 16),
 (81, 16),
 (90, 16),
 (18, 15),
 (25, 15),
 (14, 14),
 (26, 14),
 (35, 14),
 (75, 14),
 (19, 13),
 (24, 13),
 (38, 13),
 (49, 13),
 (64, 13),
 (6, 12),
 (41, 12),
 (62, 12),
 (76, 12),
 (80, 12),
 (86, 12),
 (89, 12),
 (1, 11),
 (40, 11),
 (53, 11),
 (68, 11),
 (71, 11),
 (83, 11),
 (91, 11),
 (17, 10),
 (30, 10),
 (34, 10),
 (36, 10),
 (42, 10),
 (46, 10),
 (48, 10),
 (57, 10),
 (52, 9),
 (66, 9),
 (70, 9),
 (99, 9),
 (15, 8),
 (54, 8),
 (56, 8),
 (84, 8),
 (92, 8),
 (98, 8),
 (7, 7),
 (11, 7),
 (22, 7),
 (27, 7),
 (31, 7),
 (39, 7),
 (55, 7),
 (65, 7),
 (72, 7),
 (9, 6),
 (28, 6),
 (32, 6),
 (47, 6),
 (50, 6),
 (60, 6),
 (63, 6),
 (82, 6),
 (33, 5),
 (58, 5),
 (73, 5),
 (77, 5),
 (79, 5),
 (85, 5),
 (87, 5),
 (95, 5),
 (44, 3),
 (78, 3),
 (96, 3),
 (97, 3),
 (16, 2),
 (43,

<h1>
Problem 5: A row's correct count (20 points)
</h1>
<div class=h1_cell>
<p>
For each row, count how many times it was correct. Store your result in correct_list, in sorted fashion. Here are the first part of my results.
  <pre>
[(0, 25),
 (12, 25),
 (37, 25),
 (4, 19),
 (13, 19),
 (51, 19),
 (59, 19),
 (67, 18),
 (69, 14),
 (8, 13)]
 </pre>
  So when row 0 was in the top-k, it was correct 25 times.
</div>

In [0]:
def knn3(row_index, table, columns,k, target):
  length = len(table) # get the length of the table
  distance = {} # an empty dictionary to hold the distances between the row index and all the other rows in table
  t= table[columns]
  distance = euclidean_distance(t.iloc[[row_index]], t)
  # sort the distances   
  sorted_dist = sorted(distance.items(), key=operator.itemgetter(1)) # returns a sorted list of tuples 
  
  votes = {}
  
  #gather votes from k rows 
  for key,item in enumerate(sorted_dist):
    if (key > k):
      break
    row = table.iloc[item[0]]
    vote = row[target]
    votes[item[0]] = vote
  return votes

In [96]:
participant_pairs = {} # a list  of tuples (index,times correct)
for i in range(len(small_titanic)):
  participant_pairs[i] = 0
participant_pairs
for i in range(len(small_titanic)):
  row = small_titanic.loc[i]
  targ = row[target]
  votes = knn3(i, small_titanic, usable_columns, 10, target)
  for ind,val in votes.items():
    if (val == targ):
      participant_pairs[ind]+=1
  
pairs = participant_pairs.items()
sorted(pairs, key=lambda pair: pair[1], reverse=True)

[(0, 26),
 (12, 25),
 (37, 25),
 (51, 25),
 (4, 19),
 (13, 19),
 (59, 19),
 (67, 19),
 (69, 18),
 (2, 14),
 (75, 14),
 (8, 13),
 (29, 13),
 (45, 13),
 (80, 12),
 (86, 12),
 (89, 12),
 (3, 11),
 (91, 11),
 (5, 10),
 (25, 10),
 (90, 10),
 (6, 9),
 (19, 9),
 (20, 9),
 (24, 8),
 (76, 8),
 (10, 7),
 (23, 7),
 (35, 7),
 (53, 7),
 (56, 7),
 (57, 7),
 (62, 7),
 (64, 7),
 (84, 7),
 (1, 6),
 (9, 6),
 (15, 6),
 (21, 6),
 (22, 6),
 (26, 6),
 (42, 6),
 (48, 6),
 (68, 6),
 (83, 6),
 (17, 5),
 (30, 5),
 (34, 5),
 (39, 5),
 (58, 5),
 (66, 5),
 (74, 5),
 (77, 5),
 (81, 5),
 (87, 5),
 (95, 5),
 (7, 4),
 (14, 4),
 (18, 4),
 (27, 4),
 (28, 4),
 (31, 4),
 (32, 4),
 (38, 4),
 (40, 4),
 (41, 4),
 (46, 4),
 (47, 4),
 (49, 4),
 (50, 4),
 (52, 4),
 (54, 4),
 (63, 4),
 (70, 4),
 (71, 4),
 (82, 4),
 (92, 4),
 (98, 4),
 (99, 4),
 (11, 3),
 (33, 3),
 (36, 3),
 (79, 3),
 (85, 3),
 (43, 2),
 (44, 2),
 (55, 2),
 (60, 2),
 (61, 2),
 (65, 2),
 (78, 2),
 (94, 2),
 (96, 2),
 (72, 1),
 (73, 1),
 (88, 1),
 (97, 1),
 (16, 0)

[(0, 25),
 (12, 25),
 (37, 25),
 (4, 19),
 (13, 19),
 (51, 19),
 (59, 19),
 (67, 18),
 (69, 14),
 (8, 13)]

<h1>
Problem 6: Put them together to get each row's "batting average" (10 points)
</h1>
<div class=h1_cell>
<p>
Use `top_list` and `correct_list` to compute what percentage each row was correct, i.e., divide times correct by total times in top-k. Then sort the list by highest average. When I did that I got results below taken from the slice ` [50:80]`.  I am showing my results more toward the middle to  give you a better answer to shoot for. REMEMBER: this is for items 50:80.
  <pre>
[(82, 0.6),
 (85, 0.6),
 (62, 0.5833333333333334),
 (7, 0.5714285714285714),
 (31, 0.5714285714285714),
 (48, 0.5714285714285714),
 (92, 0.5714285714285714),
 (20, 0.5625),
 (53, 0.5555555555555556),
 (66, 0.5555555555555556),
 (1, 0.5454545454545454),
 (68, 0.5454545454545454),
 (30, 0.5),
 (34, 0.5),
 (35, 0.5),
 (64, 0.5),
 (83, 0.5),
 (88, 0.5),
 (96, 0.5),
 (98, 0.5),
 (70, 0.4444444444444444),
 (99, 0.4444444444444444),
 (11, 0.42857142857142855),
 (5, 0.4),
 (21, 0.4),
 (27, 0.4),
 (40, 0.4),
 (46, 0.4),
 (71, 0.4),
 (23, 0.375)]
 </pre>
  So row 82 is batting 60%. When I divide the number of times it voted correctly (in `correct_list`) by the number of times it made the top-k (in `top_list`), it was correct .6 (60%) of the time. It has a 600 batting average!
</div>

<h3>Hint</h3>

For this problem I did not define a new function. I just figured out a way to get info out of top_list and correct_list to get what I wanted.

In [0]:
#remember that you should show rows 50:80


[(82, 0.6),
 (85, 0.6),
 (62, 0.5833333333333334),
 (7, 0.5714285714285714),
 (31, 0.5714285714285714),
 (48, 0.5714285714285714),
 (92, 0.5714285714285714),
 (20, 0.5625),
 (53, 0.5555555555555556),
 (66, 0.5555555555555556),
 (1, 0.5454545454545454),
 (68, 0.5454545454545454),
 (30, 0.5),
 (34, 0.5),
 (35, 0.5),
 (64, 0.5),
 (83, 0.5),
 (88, 0.5),
 (96, 0.5),
 (98, 0.5),
 (70, 0.4444444444444444),
 (99, 0.4444444444444444),
 (11, 0.42857142857142855),
 (5, 0.4),
 (21, 0.4),
 (27, 0.4),
 (40, 0.4),
 (46, 0.4),
 (71, 0.4),
 (23, 0.375)]

<h2>
Extra Credit: razor-thin majority
</h2>
<div class=h1_cell>
<p>
Find what target rows were decided by either 1 vote or were a tie. 
</div>

<h3>Hint</h3>

I went back to modifying knn for this problem. I replaced the knn function with a razor_thin function. It did much of the same work as knn, but instead of returning a prediction, it returned True or False. True if there was razor thin majority (or tie vote) and False otherwise. I then kept track of what rows I got a True for and sorted the list and was done.
<p>
  Caveat: I found no razor-thin votes so had an empty list. I even tried with entire titanic table and still got no razor thins. I wonder if I can put this off to "group think". Never seems to be a close call.

None
